In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S1'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #CHange from 3 to 5 as in paper
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 50
Total Tokens Found = 626
Total Tokens Output = 626
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 50
Total Tokens= 626
Unique Tokens= 252
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 50
Sorted Token Size = 252
Clean Token Size = 90
*Stop Replacements here
References Processed =  50
Total Replacement Pairs = 1
Tokens Read = 626
Tokens Changed =  3
References Changed = 3


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [5]:
import numpy as np

Create a function of the parameters that returns the F score

In [6]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [8]:
entropy_F1(mu=0.5,muIncr=0.1,beta=6,sigma=7,epsilon=4.9,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 4.9
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 108
NewMatrix Iterateblocks Total Time = 0.23098319862037897


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 4.9
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 15
NewMatrix Iterateblocks Total Time = 0.13586416095495224

Precision = TP / (TP + FP) ......................... = 1.00000000
Recall = TP / (TP + FN) ............................ = 0.96296296
F-Measure .......................................... = 0.98113208


0.98113208

Create functions for variable transformation.

In [9]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [10]:
inv_softplus(1)

0.541324854612918

In [11]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [12]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [13]:
import os, sys

In [14]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [15]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [16]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [38]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-0.7586617742112225

In [17]:
optimizer=None

In [18]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [19]:
from bayes_opt import SequentialDomainReductionTransformer

In [20]:
bounds_transformer = SequentialDomainReductionTransformer()

In [21]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [22]:
# optimizer.probe(
#     params={"beta_sp": inv_softplus(6), 
#             "sigma_sp": inv_softplus(7),
#             "mu_logit":logit(0.5),
#             #'epsilonIterate_logit': logit(0),
#             "muIncr_logit":logit(0.1),
#             "epsilon_sp":inv_softplus(4.7)},
#     lazy=True,
# )

In [23]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=5,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  1        |  3.951    |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  2        |  3.951    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  3        |  1.068    |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  4        |  124.3    |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|EpIter:0.00
|  5        |  2.442    |  560.6    |  677.8    | -3.589    |  3.181    |  613.5    |
Mu:0.69|Mu+:0.01|Beta:626.00|Sigma:614.00|Epsilon:60.03

|  39       |  124.3    |  464.1    |  297.7    | -2.819    |  0.6093   |  144.5    |
Mu:0.65|Mu+:0.06|Beta:464.00|Sigma:144.00|Epsilon:297.19|EpIter:0.00
|  40       |  124.3    |  464.6    |  297.2    | -2.822    |  0.6129   |  144.0    |
Mu:0.65|Mu+:0.06|Beta:465.00|Sigma:143.00|Epsilon:296.72|EpIter:0.00
|  41       |  124.3    |  465.1    |  296.7    | -2.82     |  0.6161   |  143.5    |
Mu:0.65|Mu+:0.06|Beta:465.00|Sigma:143.00|Epsilon:296.29|EpIter:0.00
|  42       |  124.3    |  465.5    |  296.3    | -2.819    |  0.619    |  143.1    |
Mu:0.65|Mu+:0.06|Beta:465.00|Sigma:142.00|Epsilon:295.90|EpIter:0.00
|  43       |  124.3    |  465.9    |  295.9    | -2.817    |  0.6216   |  142.7    |
Mu:0.65|Mu+:0.06|Beta:466.00|Sigma:142.00|Epsilon:295.55|EpIter:0.00
|  44       |  124.3    |  466.3    |  295.6    | -2.816    |  0.624    |  142.3    |
Mu:0.65|Mu+:0.06|Beta:466.00|Sigma:141.00|Epsilon:295.24|EpIter:0.00
|  45       |  124.3    |  466.6    |  295.2    | -2.814    |  0.6261 

In [24]:
optimizer.max

{'target': 124.33959502167846,
 'params': {'beta_sp': 469.50564125202453,
  'epsilon_sp': 292.4287890501312,
  'muIncr_logit': -2.8023816881646124,
  'mu_logit': 0.6450948082385844,
  'sigma_sp': 139.1051299542506}}

In [25]:
inv_logit(optimizer.max["target"])

1.0

In [26]:
params=optimizer.max["params"]

In [27]:
softplus(params["beta_sp"])

469.50564125202453

In [28]:
softplus(params["sigma_sp"])

139.1051299542506

In [29]:
inv_logit(params["mu_logit"])

0.6559042391621853

In [30]:
inv_logit(params["muIncr_logit"])

0.057195609570796516

In [31]:
softplus(params["epsilon_sp"])

292.4287890501312

In [32]:
entropy_F1(mu=0.656,
           muIncr=0.0572,
           beta=469,
           sigma=139,
           epsilon=292.43,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.656
Starting Iteration epsilon= 292.43
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 121
NewMatrix Iterateblocks Total Time = 0.5215561715885997

Precision = TP / (TP + FP) ......................... = 1.00000000
Recall = TP / (TP + FN) ............................ = 1.00000000
F-Measure .......................................... = 1.00000000


1.0